In [1]:
import os
import shutil

import pandas as pd
import tensorflow as tf
import numpy as np
import tensorflow_hub as hub
import tensorflow_text as text
import keras

from keras import Input
from keras.layers import TextVectorization
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.keras.layers import (
    GRU,
    Conv1D,
    Dense,
    Embedding,
    Flatten,
    Lambda,
)
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical

print(tf.__version__)

2023-02-16 14:56:46.470825: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-16 14:56:47.357631: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2023-02-16 14:56:47.357746: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/loca

2.11.0


In [2]:
bert_model = hub.KerasLayer("https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/2")
# preprocess = hub.load("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")

2023-02-16 14:56:50.273525: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-16 14:56:50.287089: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-16 14:56:50.288831: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-16 14:56:50.291080: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorF

In [3]:
bert_preprocess_model = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
# preprocess = hub.load

In [4]:
#read data
# data = pd.read_csv("../data/patent_rand100k.csv")
data = pd.read_csv("../data/patent_evan=100k.csv")
#shuffle data
data = data.sample(frac=1)

In [5]:
# # data.groupby('ipc_section').count()
# dataSort = data.query('(ipc_section == "A")').sample(n=1000)
# dataSort = pd.concat([dataSort,data.query('(ipc_section == "B")').sample(n=1000)])
# dataSort = pd.concat([dataSort,data.query('(ipc_section == "C")').sample(n=1000)])
# dataSort = pd.concat([dataSort,data.query('(ipc_section == "D")').sample(n=1000)])
# dataSort = pd.concat([dataSort,data.query('(ipc_section == "E")').sample(n=1000)])
# dataSort = pd.concat([dataSort,data.query('(ipc_section == "F")').sample(n=1000)])
# dataSort = pd.concat([dataSort,data.query('(ipc_section == "G")').sample(n=1000)])
# dataSort = pd.concat([dataSort,data.query('(ipc_section == "H")').sample(n=1000)])

# dataSort

In [6]:
#one hot encode target
# classes = [[0,1,2,3,4,5,6,7][source] for source in dataSort.ipc_section]
classes = [{"A":0,"B":1,"C":2,"D":3,"E":4,"F":5,"G":6,"H":7}[source] for source in data.ipc_section]
target = to_categorical(classes)
#print target
# target[2002]

In [7]:
BATCHSIZE = 50

inputs = Input(shape=(),dtype=tf.string,name="InputString")
x = bert_model(bert_preprocess_model(inputs))
# x = Dense(256,activation="relu",kernel_regularizer=tf.keras.regularizers.L2(0.01))(x["pooled_output"])
# x = tf.keras.layers.Dropout(0.2)(x)
# x = GRU(16)(inputs)
# x = inputs
outputs = Dense(8,activation="softmax")(x["pooled_output"])

model = keras.Model(inputs=inputs,outputs=outputs,name="modeltest")
model.summary()
model.compile(optimizer="adam", loss="categorical_crossentropy",metrics=["Accuracy"])

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Model: "modeltest"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 InputString (InputLayer)       [(None,)]            0           []                               
                                                                                                  
 keras_layer_1 (KerasLayer)     {'input_type_ids':   0           ['InputString[0][0]']            
                                (None, 128),                                                      
                                 'input_word_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                  

In [8]:
data.abstract[80000:]

83215    The invention relates to a dimethyl silicic ac...
14925    The application provides a train brake lining ...
49551    The silo filling mechanism comprises a double-...
10724    The invention relates to a rail basket for mul...
21333    PURPOSE:To provide the subject composition hav...
                               ...                        
42265    The invention discloses an intelligent bus pre...
56641    The utility model relates to the technical fie...
99165    The invention belongs to the technical field o...
22986    A kind of automatic analysing apparatus is pro...
94186    The utility model relates to an injection mold...
Name: abstract, Length: 20000, dtype: object

In [ ]:
modelhistory = model.fit(
    data.abstract[40000:50000],
    target[40000:50000], 
    epochs=10,
    batch_size=BATCHSIZE, 
    # validation_split=0.2
    validation_data=(data.abstract[50000:58000],target[50000:58000]),
    # callbacks=[EarlyStopping(patience=10)]
)

Epoch 1/10


2023-02-16 14:57:02.450888: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7f05e800f4c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-02-16 14:57:02.450927: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2023-02-16 14:57:02.457298: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-02-16 14:57:02.639818: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


200/200 [==============================] - 72s 337ms/step - loss: 1.5702 - Accuracy: 0.4017 - val_loss: 1.4221 - val_Accuracy: 0.4642
Epoch 2/10
200/200 [==============================] - 66s 331ms/step - loss: 1.3891 - Accuracy: 0.4817 - val_loss: 1.3849 - val_Accuracy: 0.4691
Epoch 3/10
200/200 [==============================] - 65s 327ms/step - loss: 1.3277 - Accuracy: 0.5018 - val_loss: 1.3161 - val_Accuracy: 0.5052
Epoch 4/10
200/200 [==============================] - 65s 327ms/step - loss: 1.2956 - Accuracy: 0.5178 - val_loss: 1.3114 - val_Accuracy: 0.5092
Epoch 5/10
200/200 [==============================] - ETA: 0s - loss: 1.2797 - Accuracy: 0.5243

In [ ]:
data.iloc[49141]

In [ ]:
pd.DataFrame(modelhistory.history)[["loss","val_loss"]].plot(title="BS="+ str(BATCHSIZE) +" | USE | 256 dense")
pd.DataFrame(modelhistory.history)[["Accuracy","val_Accuracy"]].plot(title="BS="+ str(BATCHSIZE) +" | USE | 256 dense")

In [ ]:
pd.DataFrame(modelhistory.history)[["loss","val_loss"]].plot(title="BS="+ str(BATCHSIZE) +" | USE | 256 dense")
pd.DataFrame(modelhistory.history)[["Accuracy","val_Accuracy"]].plot(title="BS="+ str(BATCHSIZE) +" | USE | 256 dense")